####Atomic Transactions

In [0]:
from pyspark.sql.functions import lit

In [0]:
(spark
   .read
     .csv("/FileStore/shared_uploads/online_retail/")
   .write
     .mode("overwrite")
     .format("parquet")
     .save("/tmp/retail.parquet")
)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3873971238932108> in <module> 
 4 . write
 5 . format ( "parquet" ) 
 ----> 6 . save ( "/tmp/retail.parquet" ) 
 7 )

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 828 self . _jwrite . save ( ) 
 829 else : 
 --> 830 self . _jwrite . save ( path ) 
 831 
 832 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : path dbfs:/tmp/retail.parquet already exists.;

In [0]:
%fs ls /tmp/retail.parquet

path,name,size
dbfs:/tmp/retail.parquet/_started_6494773441913676596,_started_6494773441913676596,0
dbfs:/tmp/retail.parquet/_started_6775149024880509486,_started_6775149024880509486,0
dbfs:/tmp/retail.parquet/part-00006-tid-6775149024880509486-a83d662e-809e-4fb7-beef-208d983f0323-212-1-c000.snappy.parquet,part-00006-tid-6775149024880509486-a83d662e-809e-4fb7-beef-208d983f0323-212-1-c000.snappy.parquet,679438
dbfs:/tmp/retail.parquet/part-00007-tid-6775149024880509486-a83d662e-809e-4fb7-beef-208d983f0323-213-1-c000.snappy.parquet,part-00007-tid-6775149024880509486-a83d662e-809e-4fb7-beef-208d983f0323-213-1-c000.snappy.parquet,601248


In [0]:
(spark
   .read
     .format("parquet")
     .load("dbfs:/tmp/retail.parquet/part-00006-tid-6775149024880509486-a83d662e-809e-4fb7-beef-208d983f0323-212-1-c000.snappy.parquet")
   .count()
)

Out[34]: 102532

In [0]:
(spark
   .read
     .csv("/FileStore/shared_uploads/online_retail/")
   .write
     .mode("overwrite")
     .format("delta")
     .save("/tmp/retail.delta")
)

In [0]:
%fs ls dbfs:/tmp/retail.delta/

path,name,size
dbfs:/tmp/retail.delta/_delta_log/,_delta_log/,0
dbfs:/tmp/retail.delta/part-00006-2fef3c0c-ce30-4d2b-8865-3e563c0d94c6-c000.snappy.parquet,part-00006-2fef3c0c-ce30-4d2b-8865-3e563c0d94c6-c000.snappy.parquet,679438
dbfs:/tmp/retail.delta/part-00007-caf3eaca-33b1-4818-b2f3-a92db7498eb6-c000.snappy.parquet,part-00007-caf3eaca-33b1-4818-b2f3-a92db7498eb6-c000.snappy.parquet,601248


In [0]:
(spark
   .read
     .format("delta")
   .load("dbfs:/tmp/retail.delta/part-00006-2fef3c0c-ce30-4d2b-8865-3e563c0d94c6-c000.snappy.parquet").count()
)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3873971238932112> in <module> 
 ----> 1 spark . read . format ( "delta" ) . load ( "dbfs:/tmp/retail.delta/part-00006-2fef3c0c-ce30-4d2b-8865-3e563c0d94c6-c000.snappy.parquet" ) . count ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 176 self . options ( ** options ) 
 177 if isinstance ( path , basestring ) : 
 --> 178 return self . _df ( self . _jreader . load ( path ) ) 
 179 elif path is not None : 
 180 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : A partition path fragment should be the form like `part1=foo/part2=bar`. The partition path: part-00006-2fef3c0c-ce30-4d2b-8865-3e563c0d94c6-c000.snappy.parquet;

####Schema Enforcement

In [0]:
%fs rm -r /tmp/customer

res17: Boolean = true

In [0]:
from pyspark.sql.functions import lit

df1 = spark.range(3).withColumn("customer_id", lit("1"))
(df1
   .write
     .format("parquet")
     .mode("overwrite")
   .save("/tmp/customer")
)
df2 = spark.range(2).withColumn("customer_id", lit(2))
(df2
   .write
     .format("parquet")
     .mode("append")
   .save("/tmp/customer")
)

In [0]:
df3 = spark.read.parquet("/tmp/customer")
df3.show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3752182148834157> in <module> 
 1 df3 = spark . read . parquet ( "/tmp/customer" ) 
 ----> 2 df3 . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 439 """
 440 if isinstance ( truncate , bool ) and truncate : 
 --> 441 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 442 else : 
 443 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o3065.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 115.0 failed 1 times, most recent failure: Lost task 0.0 in stage 115.0 (TID 480, ip-10-172-239-175.us-west-2.compute.internal, executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/tmp/customer/part-00003-tid-7892116435555669120-564a16ef-6096-4675-b607-27e78ec76f75-474-1-c000.snappy.parquet. Parquet column cannot be converted. Column: [CustomerID], Expected: StringType, Found: INT32
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:343)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:313)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:475)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:54)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:101)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.sql.execution.datasources.SchemaColumnConvertNotSupportedException
	at com.databricks.sql.io.parquet.VectorizedColumnReader.constructConvertNotSupportedException(VectorizedCol

##Delta Advantages
###Schema Validation

In [0]:
from pyspark.sql.functions import lit

df1 = spark.range(3).withColumn("customer_id", lit("1"))
(df1
   .write
     .format("delta")
     .mode("overwrite")
   .save("/tmp/delta/customer")
)
df2 = spark.range(2).withColumn("customer_id", lit(2))
(df2
   .write
     .format("delta")
     .mode("append")
   .save("/tmp/delta/customer")
)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-2566681081752889> in <module> 
 5 
 6 df2 = spark . range ( 2 ) . withColumn ( "CustomerID" , lit ( 2 ) ) 
 ----> 7 df2 . write . format ( "delta" ) . mode ( "append" ) . save ( "/tmp/delta/customer" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 828 self . _jwrite . save ( ) 
 829 else : 
 --> 830 self . _jwrite . save ( path ) 
 831 
 832 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Failed to merge fields 'CustomerID' and 'CustomerID'. Failed to merge incompatible data types StringType and IntegerType;;

###Schema Evolution

In [0]:
from pyspark.sql.functions import lit

df1 = spark.range(3)
(df1
   .write
     .format("delta")
     .mode("overwrite")
   .save("/tmp/delta/customer"))

df2 = spark.range(2).withColumn("customer_id", lit(2))
(df2
   .write
     .format("delta")
     .option("mergeSchema", True)
     .mode("append")
   .save("/tmp/delta/customer"))

+---+-----------+
 id|customer_id|
+---+-----------+
 0| 2|
 1| 2|
 0| null|
 1| null|
 2| null|
+---+-----------+

###UPDATES & DELETES

In [0]:
from pyspark.sql.functions import lit

df1 = spark.range(5).withColumn("customer_id", lit(2))
df1.write.format("delta").mode("overwrite").save("/tmp/df1")
spark.read.format("delta").load("/tmp/df1").show()

+---+-----------+
 id|customer_id|
+---+-----------+
 0| 2|
 1| 2|
 2| 2|
 3| 2|
 4| 2|
+---+-----------+

UPDATE command

In [0]:
%sql
UPDATE delta.`/tmp/delta/df1` SET customer_id = 5 WHERE id > 2;

SELECT * FROM delta.`/tmp/df1`;

id,customer_id
0,2
1,2
2,2
3,5
4,5


DELETE command

In [0]:
%sql
DELETE FROM delta.`/tmp/df1` WHERE id = 4;

SELECT * FROM delta.`/tmp/df1`;

id,customer_id
0,2
1,2
2,2
3,5


In [0]:
%fs ls /tmp/df1/_delta_log/

path,name,size
dbfs:/tmp/delta/df1/_delta_log/00000000000000000000.crc,00000000000000000000.crc,89
dbfs:/tmp/delta/df1/_delta_log/00000000000000000000.json,00000000000000000000.json,2736
dbfs:/tmp/delta/df1/_delta_log/00000000000000000001.crc,00000000000000000001.crc,89
dbfs:/tmp/delta/df1/_delta_log/00000000000000000001.json,00000000000000000001.json,1381
dbfs:/tmp/delta/df1/_delta_log/00000000000000000002.crc,00000000000000000002.crc,89
dbfs:/tmp/delta/df1/_delta_log/00000000000000000002.json,00000000000000000002.json,836


In [0]:
%sql DESCRIBE HISTORY delta.`/tmp/df1`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2021-01-15T11:38:52.000+0000,4803548200198498,snudurupati@outlook.com,DELETE,"Map(predicate -> [""(`id` = 4L)""])",null,List(3873971238932107),0115-104739-coach350,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numDeletedRows -> 1, numAddedFiles -> 1, numCopiedRows -> 0)",null
1,2021-01-15T11:10:09.000+0000,4803548200198498,snudurupati@outlook.com,UPDATE,Map(predicate -> (id#1551L > 2)),null,List(3873971238932107),0115-104739-coach350,0,WriteSerializable,false,"Map(numRemovedFiles -> 2, numAddedFiles -> 2, numUpdatedRows -> 2, numCopiedRows -> 0)",null
0,2021-01-15T10:59:17.000+0000,4803548200198498,snudurupati@outlook.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(3873971238932107),0115-104739-coach350,null,WriteSerializable,true,"Map(numFiles -> 6, numOutputBytes -> 3955, numOutputRows -> 5)",null


In [0]:
%sql SELECT * from delta.`/tmp/df1` VERSION AS OF 0

id,customer_id
0,2
1,2
2,2
3,2
4,2


Rollbacks

In [0]:
%sql 
INSERT OVERWRITE delta.`/tmp/df1`
SELECT * from delta.`/tmp/df1` VERSION AS OF 0

In [0]:
%sql SELECT * from delta.`/tmp/df1`

id,customer_id
0,2
1,2
2,2
3,2
4,2


####Clean-up 
**Databricks Community Edition** has a limitation of 10000 files and 10 GB of storage in DBFS.<br>
So it is prudent to clean-up any intermediate datasets created on DBFS that we do not intent to use at a later time.

In [0]:
%fs rm -r /tmp/